In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"));

import sys
sys.path.append('..')
from LocationsDB import LocationsDatabase

from datetime import datetime

from glob import glob

In [2]:
loc_db = LocationsDatabase('../database_files/ECON_Locations_DB.db')
# loc_db = LocationsDatabase('../database_files/testing_database.db')

In [3]:
#### 2024/18 is may ECOND packaging (RECEIVED JUNE 4)
#### 2024/29 is july packaging run (RECEIVED SEPTEMBER 1 ???)
#### 2024/31 is august run (RECEIVED SEPTEMBER 17)
data = []
for tray in range(10006,10144):
    if tray<=10011:
        lot_,date_,timestamp_ = 'N61.H30.00','2024/18',datetime.strptime('2024-06-04','%Y-%m-%d')
    elif tray<=10091:
        lot_,date_,timestamp_ = 'N61.H52.00','2024/29',datetime.strptime('2024-08-15','%Y-%m-%d')
    elif tray<=10141:
        lot_,date_,timestamp_ = 'N61.H30.00','2024/31',datetime.strptime('2024-09-17','%Y-%m-%d')
    elif tray<=10143:
        lot_,date_,timestamp_ = 'N61.H52.00','2024/29',datetime.strptime('2024-08-15','%Y-%m-%d')
    else:
        print('ISSUE HERE')
    N = 90
    if tray==10112:
        N = 50
    if tray==10121:
        continue
    if tray==10142:
        N=25
    for chip in range(1,N+1):
        chip_id = tray*100 + chip
        data.append([chip_id,tray,chip,lot_,date_,timestamp_])

In [4]:
for d in data:
    chip_id,tray,pos,pkg_batch,pkg_date,timestamp = d
    loc_db.checkin_chip(chip_id,'ECOND',tray,pos,'WH14',pkg_date,pkg_batch,'',str(timestamp))
loc_db.conn.commit()    

In [5]:
loc_db.loadLocationsDatabase().shape

(12225, 9)

In [6]:
loc_db.loadStatusDatabase().shape

(12225, 7)

In [7]:
testing_df = pd.read_csv('../testingCSV/econd_chip_test_results_2025-04-24_09-15-22.csv')


c = list(testing_df.columns)
c[0] = 'chip_id'
testing_df.columns = c

testing_df = testing_df[['chip_id','Timestamp']]

testing_df.chip_id = (testing_df.chip_id%1000000)+1000000
testing_df.Timestamp = pd.to_datetime(testing_df.Timestamp)
testing_df.sort_values('Timestamp',inplace=True)

testing_df.reset_index(inplace=True,drop=True)

testing_df.dropna(inplace=True)


In [8]:
d = loc_db.getCurrentLocations()
d.set_index('chip_id',inplace=True)
for x in testing_df.values:
    chip_id,time = x
    _tray,_pos = d.loc[chip_id][['current_tray','current_position']].values.flatten()
    loc_db.setTestedStatus(chip_id,_tray,_pos,'',str(time))
loc_db.conn.commit()

In [9]:
loc_db.loadLocationsDatabase().shape

(27011, 9)

In [10]:
loc_db.loadStatusDatabase().shape

(12225, 7)

In [11]:
d_ = []
for f in glob('../sorting_logs/ECOND_*'):
    d = pd.read_csv(f,header=None)
    d.columns = ['start_tray','start_pos','end_tray','end_pos']
    d['timestamp'] = datetime.strptime(f.split('_')[2],'%Y%m%d')
    if 'Nadja' in f:
        d['shippingLocation'] = 'UMN'
    if 'Paul' in f:
        d['shippingLocation'] = 'Baylor-Partial'### CMs get listed as Baylor for final location
    
    d_.append(d)
sorting_df = pd.concat(d_)
sorting_df['start_tray_int'] = sorting_df.start_tray.str[-5:].astype(int)

### CORRECT THE TRAY NUMBERS TO PUT THEM INTO THE APPROPRIATE SPOT IN THE TRAY SEQUENCE
sorting_df['end_tray_corrected'] = sorting_df['end_tray'].str.replace('10803','18009').str.replace('10804','18010').str.replace('10805','18011').str.replace('10806','18012')
sorting_df['end_tray_int'] = sorting_df.end_tray_corrected.str[-5:].astype(int)

sorting_df['chip_id'] = sorting_df.start_tray_int*100 + sorting_df.start_pos

sorting_df.reset_index(drop=True,inplace=True)


for idx in sorting_df.index:
    s = sorting_df.loc[idx]
    chip_id = s.chip_id
    if not 'TESTED' in loc_db.getChip(chip_id).entry_type.values:
        print(f'ISSUE WITH {chip_id}')
    else:
        loc_db.sortChip(int(chip_id),int(s.start_tray_int),int(s.start_pos),int(s.end_tray_int),int(s.end_pos),'',str(s.timestamp))
        loc_db.setChipGrade(int(chip_id),'PRESERIES','',str(s.timestamp))
loc_db.conn.commit()

In [12]:
sorting_df2 = pd.read_csv('../sorting_logs/ECOND-Minnesota-3-Dec-24.csv',header=1)
sorting_df2.chip_number = sorting_df2.chip_number+1000000
sorting_df2['start_tray'] = (sorting_df2.chip_number/100).astype(int)
sorting_df2['start_pos'] = sorting_df2.chip_number%100
sorting_df2['new_position'] = sorting_df2.new_tray_position%100
sorting_df2.reset_index(drop=True,inplace=True)

for idx in sorting_df2.index:
    s = sorting_df2.loc[idx]
    chip_id = s.chip_number
    if not 'TESTED' in loc_db.getChip(chip_id).entry_type.values:
        print(f'ISSUE WITH {chip_id}')
    else:
        loc_db.sortChip(int(chip_id),int(s.start_tray),int(s.start_pos),int(s.new_tray),int(s.new_position),'',str(datetime.strptime('2024-12-03','%Y-%m-%d')))
        loc_db.setChipGrade(int(chip_id),'PRESERIES','',str(datetime.strptime('2024-12-03','%Y-%m-%d')))
loc_db.conn.commit()

In [13]:
loc_db.loadLocationsDatabase().shape

(27384, 9)

In [14]:
loc_db.getCurrentLocations().shape

(12225, 9)

In [15]:
loc_db.getChipsInTray(18007)

,chip_id,entry_type,initial_tray,initial_position,current_tray,current_position,location,comments,time
27284,1001412,SORTED,10014,12,18007,1,WH14,,2024-12-03
27285,1001420,SORTED,10014,20,18007,2,WH14,,2024-12-03
27286,1001421,SORTED,10014,21,18007,3,WH14,,2024-12-03
27287,1001450,SORTED,10014,50,18007,4,WH14,,2024-12-03
27288,1001454,SORTED,10014,54,18007,5,WH14,,2024-12-03
...,...,...,...,...,...,...,...,...,...
27369,1001827,SORTED,10018,27,18007,86,WH14,,2024-12-03
27370,1001828,SORTED,10018,28,18007,87,WH14,,2024-12-03
27371,1001830,SORTED,10018,30,18007,88,WH14,,2024-12-03
27372,1001831,SORTED,10018,31,18007,89,WH14,,2024-12-03


In [16]:
loc_db.getChipsInTray(10014)

,chip_id,entry_type,initial_tray,initial_position,current_tray,current_position,location,comments,time
12440,1001401,TESTED,10014,1,10014,1,WH14,,2024-11-01 09:07:47
12442,1001402,TESTED,10014,2,10014,2,WH14,,2024-11-01 09:13:41
12445,1001403,TESTED,10014,3,10014,3,WH14,,2024-11-01 09:19:54
12447,1001404,TESTED,10014,4,10014,4,WH14,,2024-11-01 09:25:56
12451,1001405,TESTED,10014,5,10014,5,WH14,,2024-11-01 09:32:05
...,...,...,...,...,...,...,...,...,...
12422,1001485,TESTED,10014,85,10014,85,WH14,,2024-11-01 07:17:56
12423,1001486,TESTED,10014,86,10014,86,WH14,,2024-11-01 07:24:01
12424,1001487,TESTED,10014,87,10014,87,WH14,,2024-11-01 07:29:59
12425,1001488,TESTED,10014,88,10014,88,WH14,,2024-11-01 07:36:02
